In [1]:
from google.cloud import aiplatform

print(aiplatform.__version__)

1.43.0


In [2]:
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Image,
    Part,
)

In [3]:
from typing import Any, Callable, Optional, Tuple, Union

from google.cloud import bigquery
from vertexai.generative_models import (
    ChatSession,
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Part,
    Tool,
)
import pandas as pd

In [40]:
REGION = "us-central1"
PROJECT_ID='qwiklabs-asl-02-9dacbbe2194b'
DATASET_ID="ASL_dataset4"

## チャットセッション内にfunction calling 実装

In [6]:
class ChatAgent:
    def __init__(
        self,
        model: GenerativeModel,
        tool_handler_fn: Callable[[str, dict], Any],
        max_iterative_calls: int = 5,
    ):
        self.tool_handler_fn = tool_handler_fn
        self.chat_session = model.start_chat()
        self.max_iterative_calls = 5

    def send_message(self, message: str) -> GenerationResponse:
        response = self.chat_session.send_message(message)

        # This is None if a function call was not triggered
        if len(response.candidates[0].content.parts) == 1:
            fn_call = response.candidates[0].content.parts[0].function_call
        elif len(response.candidates[0].content.parts) == 2:
            fn_call = response.candidates[0].content.parts[0].function_call or response.candidates[0].content.parts[1].function_call

        num_calls = 0
        # Reasoning loop. If fn_call is None then we never enter this
        # and simply return the response
        while fn_call:
            if num_calls > self.max_iterative_calls:
                break

            # Handle the function call
            fn_call_response = self.tool_handler_fn(
                fn_call.name, dict(fn_call.args)
            )
            num_calls += 1

            # Send the function call result back to the model
            response = self.chat_session.send_message(
                Part.from_function_response(
                    name=fn_call.name,
                    response={
                        "content": fn_call_response,
                    },
                ),
            )

            # If the response is another function call then we want to
            # stay in the reasoning loop and keep calling functions.
            if len(response.candidates[0].content.parts) == 1:
                fn_call = response.candidates[0].content.parts[0].function_call
            elif len(response.candidates[0].content.parts) == 2:
                fn_call = response.candidates[0].content.parts[0].function_call or response.candidates[0].content.parts[1].function_call

        return response

## 関数の定義

In [36]:
get_table_info_func = FunctionDeclaration(
    name="get_table_info",
    description="""
    質問されたら最初に必ずこの関数を呼びます。そして利用可能なBigQuery テーブルとテーブルの説明、カラムとカラムの説明を取得して、ユーザーの質問に適切に回答できるようにします。
    関数の出力の形式は、各テーブル毎に、テーブルID、テーブルの説明、カラム名、カラムの説明などが辞書形式となったものをリストとして渡します。
    """,
    parameters={
        "type": "object",
        "properties": {},
    },
)

sql_query_func = FunctionDeclaration(
    name="sql_query",
    description="SQL クエリを使用して BigQuery のデータから情報を取得します。",
    parameters={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": """
                BigQuery で実行したときにユーザーの質問に回答するのに役立つ、1 行の SQL クエリ を改行なしで書いてください。 
                
                以下２点に注意してテーブルとカラムの選択をしてください。
                -関数"get_table_info"の結果の'description'にはテーブルの説明が書いてあるので、それを参考に必要なテーブルを選択してください。
                -関数"get_table_info"の結果の'schema'の中にある'name'にはカラム名、'description'にはカラムの説明が書いてあるので、それを参考に必要なカラムを選択してください。
                
                以下に注意してSQLクエリを作成してください。
                -必ず、関数"get_table_info"で取得したテーブルIDでクエリ作成してください。ただし、テーブルを指定するときは、'tableReference'の'{projectId}.{datasetId}.{tableId}'の形でテーブルIDを渡してください。
                -テーブル名を指定するときは`テーブル名`で指定してください。
                -カラム名を指定するときは`カラム名`で指定してください。
                -クエリに問題がある場合は、再度SQLを見直して正しく生成してください。
                -複数のテーブルを参照するときは`個人_企業番号`でjoinしてください。
                -日付に関するカラムのデータを操作する際は、date型に直して操作してください。
                """,
            }
        },
        "required": [
            "query",
        ],
    },
)
query_tool = Tool(
    function_declarations=[
        get_table_info_func,
        sql_query_func,
    ],
)

In [41]:
client = bigquery.Client()
tables = client.list_tables(f"{PROJECT_ID}.{DATASET_ID}")
table_names = [table.table_id for table in tables]
# print(table_names)

table_id_list=[]
for table_id in table_names:
    table_id_list.append(f"{PROJECT_ID}.{DATASET_ID}.{table_id}")
# print(table_id_list)                     

In [9]:
def get_table_info():
    bq_client = bigquery.Client()
    list_table_info=[]
    for table_id in table_id_list:
        list_table_info.append(bq_client.get_table(table_id).to_api_repr())
    return list_table_info

def sql_query(query_str: str):
    bq_client = bigquery.Client()
    try:
        # clean up query string a bit
        query_str = (
            query_str.replace("\\n", "").replace("\n", "").replace("\\", "")
        )
        # print(query_str)
        query_job = bq_client.query(query_str)
        result = query_job.result()
        result = str([dict(x) for x in result])
        return result
    except Exception as e:
        return f"Error from BigQuery Query API: {str(e)}"

In [29]:
def handle_query_fn_call(fn_name: str, fn_args: dict):
    """Handles query tool function calls."""
    print(f"Function calling: {fn_name} with args: {str(fn_args)}\n")
    
    if fn_name == "get_table_info":
        result = get_table_info()
    elif fn_name == "sql_query":
        result = sql_query(fn_args["query"])
    else:
        raise ValueError(f"Unknown function call: {fn_name}")
    
    return result

In [30]:
safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
}

In [42]:
model = GenerativeModel(
    "gemini-1.5-pro",
    tools=[query_tool],
    generation_config=GenerationConfig(temperature=0.0),
    safety_settings=safety_settings
)

In [44]:
chat = ChatAgent(model=model, tool_handler_fn=handle_query_fn_call)
# Insert an initialization prompt before the first chat to help guide model behavior and output style/format

init_prompt = """
    質問には簡潔でわかりやすい回答をお願いします。
    最初に必ず`get_table_info`関数を利用してBigQueryから利用可能なすべてのテーブルの情報を入手してください。
    BigQuery テーブルをクエリして得た情報のみを使用してください。
    情報を捏造しないでください。
    
    Question:
"""
   # 3.抽出する顧客の出力項目は、「店番号_代表ＣＩＦ」「ＣＩＦ番号_代表ＣＩＦ」「取引先名_漢字_代表ＣＩＦ」「年齢_当月末_代表ＣＩＦ」「電話番号_代表ＣＩＦ」。
# 1.テーブル名：名寄せ取引属性_日次以外のテーブルに関する情報が必要なときは、必要なテーブルのカラム名:`個人_企業番号`を紐づけてjoinしてください
# 1、クエリの書き出しは'SELECT DISTINCT T1.`担当者コード_代表ＣＩＦ` ,T1.`個人_企業番号`FROM `qwiklabs-asl-02-9dacbbe2194b.ASL_Dataset.名寄せ取引属性_日次` AS T1'とします。
# 1.最初にテーブル名：CIF別取引属性_個人ではなくテーブル名：名寄せ取引属性_日次から、SELECT DISTINCTで2つのカラム名1：個人企業番号、カラム名2:担当者コード_代表ＣＩＦをとってきてください。
# 1.最初にテーブル名：CIF別取引属性_個人ではなくテーブル名：名寄せ取引属性_日次からカラム名:個人_企業番号、だけだなくカラム名：担当者コードをとってきてください。
# 2.ATMの手数料に関する情報が必要なときは、{テーブル名:'チャネル別利用状況',カラム名:`個人_企業番号`}を紐づけてjoinしてください
# 2.抽出する顧客の出力項目は、「店番号_代表ＣＩＦ」「ＣＩＦ番号_代表ＣＩＦ」「取引先名_漢字_代表ＣＩＦ」「年齢_当月末_代表ＣＩＦ」「電話番号_代表ＣＩＦ」。
prompt = """
ちばぎんアプリ利用先の一覧がほしいです
"""
response = chat.send_message(init_prompt + prompt)
print(response.text)

Function calling: get_table_info with args: {}

Function calling: sql_query with args: {'query': 'SELECT DISTINCT `取引先名_漢字_代表ＣＩＦ` FROM `qwiklabs-asl-02-9dacbbe2194b.ASL_dataset4.名寄せ取引属性_日次` WHERE `ちばぎんアプリ` = 1'}

ちばぎんアプリの利用先は人間5、人間13、人間17、人間20、人間37、人間2、人間10、人間19、株式会社15、株式会社9、株式会社12、株式会社30、人間34、人間8、人間16、人間38、株式会社18です。 



In [ ]:
response.candidates[0].content.parts[0].function_call or response.candidates[0].content.parts[1].function_call

In [88]:
chat.chat_session.history

[role: "user"
 parts {
   text: "\n    \350\263\252\345\225\217\343\201\253\343\201\257\347\260\241\346\275\224\343\201\247\343\202\217\343\201\213\343\202\212\343\202\204\343\201\231\343\201\204\345\233\236\347\255\224\343\202\222\343\201\212\351\241\230\343\201\204\343\201\227\343\201\276\343\201\231\343\200\202\n    \346\234\200\345\210\235\343\201\253\345\277\205\343\201\232`get_table_info`\351\226\242\346\225\260\343\202\222\345\210\251\347\224\250\343\201\227\343\201\246BigQuery\343\201\213\343\202\211\345\210\251\347\224\250\345\217\257\350\203\275\343\201\252\343\201\231\343\201\271\343\201\246\343\201\256\343\203\206\343\203\274\343\203\226\343\203\253\343\201\256\346\203\205\345\240\261\343\202\222\345\205\245\346\211\213\343\201\227\343\201\246\343\201\217\343\201\240\343\201\225\343\201\204\343\200\202\n    BigQuery \343\203\206\343\203\274\343\203\226\343\203\253\343\202\222\343\202\257\343\202\250\343\203\252\343\201\227\343\201\246\345\276\227\343\201\237\346\203\205\345

In [86]:
# response = chat.send_message(
#     "あります"
# )
# print(response.text)

Function calling: sql_query with args: {'query': 'SELECT DISTINCT `店番号_代表ＣＩＦ`, `ＣＩＦ番号_代表ＣＩＦ`, `取引先名_漢字_代表ＣＩＦ`, `年齢_当月末_代表ＣＩＦ`, `電話番号_代表ＣＩＦ` FROM `qwiklabs-asl-02-9dacbbe2194b.ASL_dataset3.名寄せ取引属性_日次` WHERE `店番号_代表ＣＩＦ` = 1 AND `ちばぎんアプリ` = 1'}

中央支店のちばぎんアプリ登録先一覧は下記の通りです。

店番号_代表ＣＩＦ|ＣＩＦ番号_代表ＣＩＦ|取引先名_漢字_代表ＣＩＦ|年齢_当月末_代表ＣＩＦ|電話番号_代表ＣＩＦ
------- | -------- | -------- | -------- | --------
1 | 210002 | 人間2 | 50 | 000-000-002
1 | 210005 | 人間5 | 26 | 000-000-005
1 | 210008 | 人間8 | 32 | 000-000-008
1 | 210009 | 株式会社9 | 999 | 000-000-009
1 | 210010 | 人間10 | 21 | 000-000-010
1 | 210012 | 株式会社12 | 999 | 000-000-012
1 | 210013 | 人間13 | 65 | 000-000-013
1 | 210015 | 株式会社15 | 999 | 000-000-015
1 | 210016 | 人間16 | 25 | 000-000-016
1 | 210017 | 人間17 | 53 | 000-000-017
1 | 210018 | 株式会社18 | 999 | 000-000-018
1 | 210019 | 人間19 | 43 | 000-000-019
1 | 210020 | 人間20 | 85 | 000-000-020
1 | 210030 | 株式会社30 | 999 | 000-000-030
1 | 210034 | 人間34 | 17 | 000-000-034
1 | 210037 | 人間37 | 38 | 000-000-037
1 | 210038 | 人

In [ ]:
%%bigquery
SELECT `取引先名_漢字`, `店番号`, `CIF番号` FROM `qwiklabs-asl-02-9dacbbe2194b.ASL_Dataset.CIF別取引属性_個人` WHERE `取引先名_漢字` LIKE "%人間%"

In [226]:
chat.chat_session.history

[role: "user"
 parts {
   text: "\n    \350\263\252\345\225\217\343\201\253\343\201\257\347\260\241\346\275\224\343\201\247\343\202\217\343\201\213\343\202\212\343\202\204\343\201\231\343\201\204\345\233\236\347\255\224\343\202\222\343\201\212\351\241\230\343\201\204\343\201\227\343\201\276\343\201\231\343\200\202\n    BigQuery \343\203\206\343\203\274\343\203\226\343\203\253\343\202\222\343\202\257\343\202\250\343\203\252\343\201\227\343\201\246\345\276\227\343\201\237\346\203\205\345\240\261\343\201\256\343\201\277\343\202\222\344\275\277\347\224\250\343\201\227\343\201\246\343\201\217\343\201\240\343\201\225\343\201\204\343\200\202\n    \346\203\205\345\240\261\343\202\222\346\215\217\351\200\240\343\201\227\343\201\252\343\201\204\343\201\247\343\201\217\343\201\240\343\201\225\343\201\204\343\200\202\343\202\257\343\202\250\343\203\252\343\202\222\344\275\234\346\210\220\343\201\231\343\202\213\345\211\215\343\201\253\343\200\201\345\277\205\343\201\232\343\201\251\343\201\256\343

In [141]:
response = chat.send_message("""

"""
)
print(response.text)

ResponseValidationError: The model response did not completed successfully.
Finish reason: 3.
Finish message: .
Safety ratings: [category: HARM_CATEGORY_HATE_SPEECH
probability: NEGLIGIBLE
, category: HARM_CATEGORY_DANGEROUS_CONTENT
probability: MEDIUM
blocked: true
, category: HARM_CATEGORY_HARASSMENT
probability: NEGLIGIBLE
, category: HARM_CATEGORY_SEXUALLY_EXPLICIT
probability: NEGLIGIBLE
].
To protect the integrity of the chat session, the request and response were not added to chat history.
To skip the response validation, specify `model.start_chat(response_validation=False)`.
Note that letting blocked or otherwise incomplete responses into chat history might lead to future interactions being blocked by the service.